In [1]:


print(ask_question_gpt("hey"))


Hello! How can I assist you today?


In [16]:
import json
import random
import sys
import os

# We adopted the LLM-based automatic pairwise comparison framework to assess the annotation quality of our two datasets. 
# Following prior work, we used GPT-4o to automatically evaluate the quality of the dataset annotations. 
# To mitigate order bias in the pairwise comparisons, we randomly arranged the order of samples from the two datasets. 
# The evaluation was based on three criteria: Completeness, Relevance, and Utility. 
# This approach ensures a fair and unbiased assessment of the datasets. 
# Based on this framework, we evaluated the quality of the natural language responses in both datasets

sys.path.append('../')

from QAs_generate.tools.llm import ask_question_gpt
# Define file paths
cosql_file_path = 'train_sets/CoSQL_train.json'
qdasql_file_path = 'train_sets/QDASQL_train.json'

# Initialize lists to store QA pairs
qa_cosql = []
qa_qdasql = []

# Read CoSQL_train.json file
with open(cosql_file_path, 'r', encoding='utf-8') as f:
    cosql_data = json.load(f)

# Process CoSQL data
for entry in cosql_data:
    turns = entry.get('turns', [])
    for i, turn in enumerate(turns):
        type_field = turn.get('type', '')
        if type_field in ['AMBIGUOUS', 'CANNOT_ANSWER']:
            question = turn.get('text', '')
            if i + 1 < len(turns):
                answer = turns[i + 1].get('text', '')
                qa_cosql.append({'question': question, 'answer': answer})

# Read QDASQL_train.json file
with open(qdasql_file_path, 'r', encoding='utf-8') as f:
    qdasql_data = json.load(f)

# Process QDASQL data
for entry in qdasql_data:
    turns = entry.get('turns', [])
    for i, turn in enumerate(turns):
        type_field = turn.get('type', '')
        if type_field in ['unanswerable', 'ambiguous']:
            question = turn.get('text', '')
            if i + 1 < len(turns):
                answer = turns[i + 1].get('text', '')
                qa_qdasql.append({'question': question, 'answer': answer})

# Print the number of QA pairs collected
print(f"Number of CoSQL QA pairs: {len(qa_cosql)}")
print(f"Number of QDASQL QA pairs: {len(qa_qdasql)}")

# Initialize win/loss/tie counters
wins = 0
losses = 0
ties = 0

# Perform 10 random selections
for i in range(10):
    if i % 2 == 0:
        # Even iterations, qa_qdasql goes first
        print("QDA-SQL as first")
        first = qa_qdasql
        second = qa_cosql
    else:
        # Odd iterations, qa_cosql goes first
        print("CoSQL as first")
        first = qa_cosql
        second = qa_qdasql

    # Randomly select one QA pair from each list
    if first and second:  # Ensure both lists are not empty
        selected_first = random.choice(first)
        selected_second = random.choice(second)
        
        # Print the selected QA pairs
        print(f"\nIteration {i + 1}:")
        print(f"QA Pair A - Question: {selected_first['question']}, Answer: {selected_first['answer']}")
        print(f"QA Pair B - Question: {selected_second['question']}, Answer: {selected_second['answer']}")

    # Compare lengths
    if len(first) > len(second):
        wins += 1
    elif len(first) < len(second):
        losses += 1
    else:
        ties += 1

# Print win/loss/tie results
print(f"Win: {wins}, Loss: {losses}, Tie: {ties}")


Number of CoSQL QA pairs: 1185
Number of QDASQL QA pairs: 14582
QDA-SQL as first

Iteration 1:
QA Pair A - Question: What is the average rating of items that have at least one review with a rank greater than 3 and an average rating greater than 5?, Answer: The database does not include the average rating of items, so I cannot answer this question.
QA Pair B - Question: What is the Physician name of the following Employee ID?  tell me the SSN., Answer: Sorry what employee IDs are you talking about?
CoSQL as first

Iteration 2:
QA Pair A - Question: Hmm, I want to know which friends are above 40 years old or below 30 years old. , Answer: do you mean the name of the person who has friends with age above 40 and under age 30?
QA Pair B - Question: What is the total number of degrees awarded by CSU campuses after 2000?, Answer: Do you mean the total number of degrees awarded by CSU campuses in Orange County after 2000?
QDA-SQL as first

Iteration 3:
QA Pair A - Question: What's the name of t